In [1]:
import cv2
import numpy as np
import os
import json

In [2]:
def load_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def find_largest_bbox(bbox_list):
    largest_bbox = None
    max_area = 0

    if bbox_list==[]:
        return [0,0,0,0]

    for bbox in bbox_list:
        # Calculate area of the bounding box
        area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])

        # Update largest_bbox if current area is greater
        if area > max_area:
            max_area = area
            largest_bbox = bbox

    return largest_bbox

def calculate_iou(box1, box2):
    # Get coordinates of intersection rectangle
    x_min = max(box1[0], box2[0])
    y_min = max(box1[1], box2[1])
    x_max = min(box1[2], box2[2])
    y_max = min(box1[3], box2[3])

    # Calculate area of intersection rectangle
    intersection_area = max(0, x_max - x_min + 1) * max(0, y_max - y_min + 1)

    # Calculate area of both bounding boxes
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    #if box1_area == 1 or box2_area == 1:
    #   return 0

    # Calculate IOU
    iou = intersection_area / float(box1_area + box2_area - intersection_area)

    return iou

def iou_comparision_of_faces(folder_path, iou_threshold = 0.5):
    # 첫 번째 json 파일 로드 및 가장 큰 bbox 검출
    jsons = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.json')])
    prev_bbox = load_json(jsons[0])
    prev_faces = len(prev_bbox['bbox'])
    prev_largest_face = find_largest_bbox(prev_bbox['bbox'])

    faces_list = []

    # json 파일 리스트를 순회하며 IoU 비교
    for idx,  json_path in enumerate(jsons[1:], 1):
        current_json = load_json(json_path)
        curr_largest_face = find_largest_bbox(current_json['bbox'])
        iou = calculate_iou(prev_largest_face, curr_largest_face)
        if iou < iou_threshold:
            faces_list.append(idx)

        prev_largest_face = curr_largest_face

    return faces_list

In [3]:
"""
def calculate_histogram(image, bins=256):
    # 그레이스케일로 변환
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 히스토그램 계산
    histogram = cv2.calcHist([gray_image], [0], None, [bins], [0, 256])
    # 히스토그램을 정규화
    histogram = cv2.normalize(histogram, histogram).flatten()
    return histogram

def shot_boundary_detection_images(folder_path, threshold=0.7):
    images = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')])
    if not images:
        print("No images found in the folder.")
        return

    # 첫 번째 이미지 로드 및 히스토그램 계산
    prev_image = cv2.imread(images[0])
    prev_hist = calculate_histogram(prev_image)

    images_list = []
    # 이미지 파일 리스트를 순회하며 히스토그램 비교
    for idx, image_path in enumerate(images[1:], 1):
        current_image = cv2.imread(image_path)
        curr_hist = calculate_histogram(current_image)

        # 히스토그램 비교 (비교 기법: Correlation)
        similarity = cv2.compareHist(prev_hist, curr_hist, cv2.HISTCMP_CORREL)
        if similarity < threshold:
            images_list.append(idx)
            #print(f"Shot boundary probably detected at frame {idx}")

        prev_hist = curr_hist

    return images_list
"""

'\ndef calculate_histogram(image, bins=256):\n    # 그레이스케일로 변환\n    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\n    # 히스토그램 계산\n    histogram = cv2.calcHist([gray_image], [0], None, [bins], [0, 256])\n    # 히스토그램을 정규화\n    histogram = cv2.normalize(histogram, histogram).flatten()\n    return histogram\n\ndef shot_boundary_detection_images(folder_path, threshold=0.7):\n    images = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(\'.jpg\')])\n    if not images:\n        print("No images found in the folder.")\n        return\n\n    # 첫 번째 이미지 로드 및 히스토그램 계산\n    prev_image = cv2.imread(images[0])\n    prev_hist = calculate_histogram(prev_image)\n\n    images_list = []\n    # 이미지 파일 리스트를 순회하며 히스토그램 비교\n    for idx, image_path in enumerate(images[1:], 1):\n        current_image = cv2.imread(image_path)\n        curr_hist = calculate_histogram(current_image)\n\n        # 히스토그램 비교 (비교 기법: Correlation)\n        similarity = cv2.compareHist(prev_his

In [4]:
#rgb채널 모두 활용
def calculate_histograms(image, bins=256):
    # 각 채널(B, G, R)에 대한 히스토그램 계산
    channels = cv2.split(image)
    histograms = [cv2.calcHist([ch], [0], None, [bins], [0, 256]) for ch in channels]
    # 각 히스토그램을 정규화
    histograms = [cv2.normalize(hist, hist).flatten() for hist in histograms]
    return histograms

def shot_boundary_detection_images(folder_path, threshold=0.7):
    images = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg')])
    if not images:
        print("No images found in the folder.")
        return

    # 첫 번째 이미지 로드 및 히스토그램 계산
    prev_image = cv2.imread(images[0])
    prev_hists = calculate_histograms(prev_image)

    images_list = []
    # 이미지 파일 리스트를 순회하며 히스토그램 비교
    for idx, image_path in enumerate(images[1:], 1):
        current_image = cv2.imread(image_path)
        curr_hists = calculate_histograms(current_image)

        # 각 채널 히스토그램 비교 (비교 기법: Correlation)
        similarities = [cv2.compareHist(prev_hists[i], curr_hists[i], cv2.HISTCMP_CORREL) for i in range(3)]
        if min(similarities) < threshold:
            images_list.append(idx)
            print(f"Shot boundary probably detected at frame {idx}")

        prev_hists = curr_hists

    return images_list

In [5]:
def shot_boundary_detection_images_with_faces(images_list, num_faces_list, folder_path):
    shot_boundaries = []
    for frame_idx in images_list:
        if frame_idx in num_faces_list:
            framename = f"frame_{frame_idx:05}.jpg"
            frame_img_path = os.path.join(folder_path, framename)
            shot_boundaries.append((os.path.split(folder_path)[-1], framename))
            print(f"Shot boundary detected at frame {frame_idx}")

    return shot_boundaries

In [6]:
def generate_frame_names(foldername, filename, max_frame, frame_range): #shot boundary가 주어지면 앞뒤로 frame_range만큼의 key값을 생성하기 위해 사용.
    # 파일명에서 숫자 부분 추출
    base_name = filename.split('.')[0]
    prefix = base_name.split('_')[0]
    frame_number = int(base_name.split('_')[1])

    # 생성할 프레임 번호 계산
    frame_numbers = range(max(0, frame_number - frame_range),
                          min(max_frame, frame_number + frame_range) + 1)

    # 새로운 파일명 리스트 생성
    frame_filenames = [(foldername, f"{prefix}_{str(num).zfill(5)}.jpg") for num in frame_numbers]

    return frame_filenames

def generate_path_for_delete(frame_folder_path, bbox_folder_path, max_frame, hist_threshold, iou_threshold, frame_range): #frame_range: shot boundary 앞뒤로 삭제할 frame 갯수 ex)30 -> 앞뒤로 30개씩 삭제
    total_shot_boundaries = []
    subfolders = [f for f in os.listdir(frame_folder_path) if os.path.isdir(os.path.join(frame_folder_path, f))]
    for subfolder in subfolders:
        frame_subfolder_path = os.path.join(frame_folder_path, subfolder)
        bbox_subfolder_path = os.path.join(bbox_folder_path, subfolder)

        shot_boundaries_by_histogram = shot_boundary_detection_images(frame_subfolder_path, hist_threshold) #히스토그램 이용한 샷 바운더리 검출
        shot_boundaries_by_num_faces = iou_comparision_of_faces(bbox_subfolder_path, iou_threshold) #검출된 얼굴의 IoU를 활용한 샷 바운더리 검출
        shot_boundaries1 = shot_boundary_detection_images_with_faces(shot_boundaries_by_histogram, shot_boundaries_by_num_faces, frame_subfolder_path) # 위 두 정보를 활용하여 얻은 최종 샷 바운더리
        total_shot_boundaries.extend(shot_boundaries1)

    path_for_delete = [] # 샷 바운더리를 포함하여 샷 바운더리 앞 뒤로 frame_range만큼의
    for path in total_shot_boundaries:
        folder_name = path[0]
        file_name = path[1]
        frames_near_shot_boundary = generate_frame_names(folder_name, file_name, max_frame, frame_range)
        path_for_delete.extend(frames_near_shot_boundary)

    return total_shot_boundaries, list(set(path_for_delete))

In [7]:
frame_path = 'output_frames'
bbox_path = 'bbox_data'
#max_frame_num = len(frames) - 1
max_frame_num = len(os.listdir('output_frames/1_frames/')) - 1

total_shot_boundaries, delete_list = generate_path_for_delete(frame_path, bbox_path, max_frame_num, 0.7, 0.5, 20) #15 = (11//2) + 9 (하이퍼파라미터)

Shot boundary probably detected at frame 75
Shot boundary probably detected at frame 121
Shot boundary probably detected at frame 183
Shot boundary probably detected at frame 262
Shot boundary probably detected at frame 507
Shot boundary probably detected at frame 522
Shot boundary probably detected at frame 523
Shot boundary probably detected at frame 536
Shot boundary probably detected at frame 566
Shot boundary probably detected at frame 596
Shot boundary probably detected at frame 627
Shot boundary probably detected at frame 728
Shot boundary probably detected at frame 740
Shot boundary probably detected at frame 757
Shot boundary probably detected at frame 788
Shot boundary probably detected at frame 818
Shot boundary probably detected at frame 849
Shot boundary probably detected at frame 877
Shot boundary probably detected at frame 929
Shot boundary probably detected at frame 955
Shot boundary probably detected at frame 1089
Shot boundary probably detected at frame 1184
Shot boun

In [8]:
#rgb + iou + hist_thresh 0.7
for file in total_shot_boundaries:
    print(file)

('1_frames', 'frame_00075.jpg')
('1_frames', 'frame_00121.jpg')
('1_frames', 'frame_00183.jpg')
('1_frames', 'frame_00262.jpg')
('1_frames', 'frame_00596.jpg')
('1_frames', 'frame_00627.jpg')
('1_frames', 'frame_00740.jpg')
('1_frames', 'frame_00757.jpg')
('1_frames', 'frame_00788.jpg')
('1_frames', 'frame_00818.jpg')
('1_frames', 'frame_00849.jpg')
('1_frames', 'frame_00929.jpg')
('1_frames', 'frame_00955.jpg')
('1_frames', 'frame_01184.jpg')
('1_frames', 'frame_01262.jpg')
('1_frames', 'frame_01291.jpg')
('1_frames', 'frame_01391.jpg')
('1_frames', 'frame_01406.jpg')
('1_frames', 'frame_01481.jpg')
('1_frames', 'frame_01845.jpg')
('1_frames', 'frame_01888.jpg')
('1_frames', 'frame_01955.jpg')
('1_frames', 'frame_01968.jpg')
('1_frames', 'frame_01998.jpg')
('1_frames', 'frame_02008.jpg')
('1_frames', 'frame_02147.jpg')
('1_frames', 'frame_02167.jpg')
('1_frames', 'frame_02204.jpg')
('1_frames', 'frame_02205.jpg')
('1_frames', 'frame_02245.jpg')
('1_frames', 'frame_02265.jpg')
('1_fram

In [9]:
#rgb + iou + hist_thresh 0.7
for file in delete_list:
    print(file)

('1_frames', 'frame_01956.jpg')
('1_frames', 'frame_05263.jpg')
('2_frames', 'frame_04568.jpg')
('1_frames', 'frame_02016.jpg')
('2_frames', 'frame_04569.jpg')
('1_frames', 'frame_03260.jpg')
('1_frames', 'frame_03524.jpg')
('2_frames', 'frame_01294.jpg')
('1_frames', 'frame_01289.jpg')
('2_frames', 'frame_00808.jpg')
('1_frames', 'frame_02174.jpg')
('1_frames', 'frame_01195.jpg')
('1_frames', 'frame_05196.jpg')
('1_frames', 'frame_05324.jpg')
('2_frames', 'frame_04596.jpg')
('1_frames', 'frame_05164.jpg')
('1_frames', 'frame_02770.jpg')
('1_frames', 'frame_03521.jpg')
('1_frames', 'frame_02500.jpg')
('1_frames', 'frame_05001.jpg')
('1_frames', 'frame_01903.jpg')
('1_frames', 'frame_05535.jpg')
('1_frames', 'frame_02000.jpg')
('2_frames', 'frame_02888.jpg')
('1_frames', 'frame_03841.jpg')
('1_frames', 'frame_05111.jpg')
('1_frames', 'frame_03601.jpg')
('1_frames', 'frame_03944.jpg')
('1_frames', 'frame_02635.jpg')
('1_frames', 'frame_05191.jpg')
('1_frames', 'frame_03358.jpg')
('1_fram

In [10]:
#rgb + iou + hist_thresh 0.7
print(total_shot_boundaries)

[('1_frames', 'frame_00075.jpg'), ('1_frames', 'frame_00121.jpg'), ('1_frames', 'frame_00183.jpg'), ('1_frames', 'frame_00262.jpg'), ('1_frames', 'frame_00596.jpg'), ('1_frames', 'frame_00627.jpg'), ('1_frames', 'frame_00740.jpg'), ('1_frames', 'frame_00757.jpg'), ('1_frames', 'frame_00788.jpg'), ('1_frames', 'frame_00818.jpg'), ('1_frames', 'frame_00849.jpg'), ('1_frames', 'frame_00929.jpg'), ('1_frames', 'frame_00955.jpg'), ('1_frames', 'frame_01184.jpg'), ('1_frames', 'frame_01262.jpg'), ('1_frames', 'frame_01291.jpg'), ('1_frames', 'frame_01391.jpg'), ('1_frames', 'frame_01406.jpg'), ('1_frames', 'frame_01481.jpg'), ('1_frames', 'frame_01845.jpg'), ('1_frames', 'frame_01888.jpg'), ('1_frames', 'frame_01955.jpg'), ('1_frames', 'frame_01968.jpg'), ('1_frames', 'frame_01998.jpg'), ('1_frames', 'frame_02008.jpg'), ('1_frames', 'frame_02147.jpg'), ('1_frames', 'frame_02167.jpg'), ('1_frames', 'frame_02204.jpg'), ('1_frames', 'frame_02205.jpg'), ('1_frames', 'frame_02245.jpg'), ('1_frame

In [11]:
#rgb + iou + hist_thresh 0.7
print(delete_list)

[('1_frames', 'frame_01956.jpg'), ('1_frames', 'frame_05263.jpg'), ('2_frames', 'frame_04568.jpg'), ('1_frames', 'frame_02016.jpg'), ('2_frames', 'frame_04569.jpg'), ('1_frames', 'frame_03260.jpg'), ('1_frames', 'frame_03524.jpg'), ('2_frames', 'frame_01294.jpg'), ('1_frames', 'frame_01289.jpg'), ('2_frames', 'frame_00808.jpg'), ('1_frames', 'frame_02174.jpg'), ('1_frames', 'frame_01195.jpg'), ('1_frames', 'frame_05196.jpg'), ('1_frames', 'frame_05324.jpg'), ('2_frames', 'frame_04596.jpg'), ('1_frames', 'frame_05164.jpg'), ('1_frames', 'frame_02770.jpg'), ('1_frames', 'frame_03521.jpg'), ('1_frames', 'frame_02500.jpg'), ('1_frames', 'frame_05001.jpg'), ('1_frames', 'frame_01903.jpg'), ('1_frames', 'frame_05535.jpg'), ('1_frames', 'frame_02000.jpg'), ('2_frames', 'frame_02888.jpg'), ('1_frames', 'frame_03841.jpg'), ('1_frames', 'frame_05111.jpg'), ('1_frames', 'frame_03601.jpg'), ('1_frames', 'frame_03944.jpg'), ('1_frames', 'frame_02635.jpg'), ('1_frames', 'frame_05191.jpg'), ('1_frame

In [12]:
print(len(delete_list), len(total_shot_boundaries))

4229 124
